In this file we solve 

$  - \operatorname{div}( 2 \mu \varepsilon(u)) + \lambda \nabla \operatorname{div} u = f $

on a bounded domain $\Omega$ with (pure) Dirichlet boundary data.

First, we import ngsolve and Netgens 2D geometry library:

In [ ]:
from ngsolve import *
from ngsolve.meshes import *
from draw import Draw

generate a structured triangular mesh

In [ ]:
N=0 # refinement level
mesh = MakeStructured2DMesh(quads=False,nx=2**(N+1),ny=2**(N+1))
Draw(mesh)

parameters and exact solution:

In [ ]:
mu = 1
lam = 1
order = 1

uexact = CoefficientFunction((sin(pi*x)*sin(pi*y),
                              cos(pi*x)*cos(pi*y)))
duexact = CoefficientFunction(( pi*cos(pi*x)*sin(pi*y),
                                pi*sin(pi*x)*cos(pi*y),
                               -pi*sin(pi*x)*cos(pi*y),
                               -pi*cos(pi*x)*sin(pi*y)),dims=(2,2))
coef_f = 2*mu*CoefficientFunction((pi**2*sin(pi*x)*sin(pi*y),pi**2*cos(pi*x)*cos(pi*y)))

Setup the discrete system:

In [ ]:
V = VectorH1(mesh,order=order, dirichlet="left|right|top|bottom")
u,v = V.TnT()

a = BilinearForm(V)
a += 2*mu*InnerProduct(Sym(grad(u)), Sym(grad(v))) * dx\
    + lam*div(u)*div(v) * dx
        
f = LinearForm(V)   
f += coef_f * v * dx(bonus_intorder=10)
        
a.Assemble()
f.Assemble()

Solve the linear system:

In [ ]:
# gridfunction for the solution
gfu = GridFunction(V)
gfu.Set(uexact, definedon=mesh.Boundaries("left|right|top|bottom"))
f.vec.data -= a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(V.FreeDofs()) * f.vec

Plot the solution:

In [ ]:
# plot the solution
Draw (0.05*gfu,mesh,"u",sd=1)

Compute different error measures:

In [ ]:
errorl2=sqrt(Integrate(Norm(gfu-uexact)**2,mesh,order=2*order))
errorh1=sqrt(Integrate(Norm(grad(gfu)-duexact)**2,mesh,order=2*order))
print("             L2 norm Error =", errorl2)
print("         H1 seminorm Error =", errorh1)